In [3]:
import json
from public_data.models import Ocsge, Commune
from django.db import connection

qs = Commune.objects.all()
qs





In [ ]:
comm = qs[3]
bbox = comm.mpoly.extent

# Initial parameters
# bbox = [-0.601487594874066, 44.6932226204681, -1.02920400564268, 44.4183678096998]
# generate initial SQL
query = (
    "SELECT id, couverture_label, usage_label, millesime, map_color, year, st_AsGeoJSON(mpoly, 6, 0) AS geojson " 
    f"FROM {Ocsge._meta.db_table} "
    "WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) "
    "LIMIT 2"
)

query

In [ ]:
with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

In [ ]:
# generate features
features = []
for row in rows:
    feature = {
        'type': 'Feature',
        'properties': {
            'id': row[0],
            'couverture_label': row[1],
            'usage_label': row[2],
            'millesime': row[3].strftime("%Y-%m-%d"),
            'map_color': row[4],
            'year': row[5],
        },
        'geometry': json.loads(row[6])
    }
    features.append(feature)
features

In [ ]:
# convert python dictionary into json
geojson = json.dumps({
    "type": "FeatureCollection",
    "crs": {"type": "name", "properties": {"name": f"EPSG:4326"}},
    'features': features
})

print(geojson)

In [ ]:


qs = CommunesSybarval.objects.all()
qs = qs.annotate(extent=Extent("mpoly"))
comm = qs.first()
print(comm.extent)
print(comm.mpoly.extent)

bbox = [-0.601487594874066, 44.6932226204681, -1.02920400564268, 44.4183678096998]

bbox = [comm.extent[2], comm.extent[3], comm.extent[0], comm.extent[1]]
print(bbox)

In [ ]:
# test avec jointure sur UsageSol
params = [-1.2747573852539065, 44.609534650700205, -1.0524559020996096, 44.69062362134582, 2015]

query = (
    "SELECT o.id, o.couverture_label, o.usage_label, o.millesime, u.map_color, "
    "o.year, st_AsGeoJSON(o.mpoly, 4) AS geojson "
    f"FROM {Ocsge._meta.db_table} o "
    f"INNER JOIN {UsageSol._meta.db_table} u ON u.code_prefix = o.usage "
    "WHERE o. mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) "
    "AND o.year = %s"
)

print(query)

with connection.cursor() as cursor:
    cursor.execute(query, params)
    rows = cursor.fetchall()

rows[:5]

In [ ]:
from public_data.models import RefPlan
table_name = RefPlan._meta.db_table
query = (
    "SELECT region_id, region_name, "
    "st_AsGeoJSON(ST_Union(mpoly), 8) AS geojson "
    f"FROM {table_name} "
    "GROUP BY region_id, region_name"
)
query

In [ ]:
with connection.cursor() as cursor:
    cursor.execute(query)
    rows = cursor.fetchall()
len(rows)